## 시설물 기본제원

In [1]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "시설물 기본제원"
### APIKEY 불러오기 ###
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/시설물 기본제원/selectIoFmSpcfDtl.csv'

In [2]:
APIKEYLEN = len(APIKEY)

In [3]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectIoFmSpcfDtl'

In [4]:
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
REQPARAM

['serviceKey', 'fcno', 'type']

In [5]:
JSONKEY = "detail1"
DUMMY = 0

In [6]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/시설물 목록/selectIoFmMngList.csv", encoding="ms949")

In [7]:
dfCombi = imsiDf.loc[:,["fcno"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombi

,fcno
0,017F251FED9EB079AB33F9C412F5B472
1,017F252A645BC823DA30560A8759FD30
2,017F2529B176518CD178B7E35156C846
3,017F25293BAD9CF86706B714FA809DA2
4,017F2528B4559D9301445A72A86CA350
...,...
7910,0175BC9111C42D0A7B6872462096F666
7911,0175BC910E164DC7DADE7C66200B25E9
7912,0175BC9118FF6D756E9A835EBFA810BE
7913,0175BC911A6DC7E714F3234410E50628


In [8]:
newParam = dfCombi.fcno.drop_duplicates().tolist()

In [9]:
orgParam = []
try:
    # 기존 데이터 및 파라미터 정보 불러오기
    # 리스트로 불러오기로 변경 0425
    orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)[0]
## 파일이 없는경우
except Exception as e:
    orgParam = []
    print(e)

ptlcmnoList = list( set(newParam) - set(orgParam) )
# 모드: 0=종료 1=append 2=새로생성
mode = 2

# 업데이트 할 내용이 없으면 종료
if ptlcmnoList == []:
    mode = 0
    print("{} 정보 quit모드 {} ".format(DATANAME, mode))
    quit()
# orgParam == []
elif orgParam == []:
    mode = 2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))
# 이외에는 append 모드
else:
    mode = 1
    print("{} 정보 append 모드 {} ".format(DATANAME, mode))

[Errno 2] No such file or directory: '../output/건설사업정보시스템/시설물 기본제원/selectIoFmSpcfDtl.pickle'
시설물 기본제원 정보 new 모드 2 


## 시설물 기본전용 전용 스크랩 함수

In [19]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os
import time

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn,inAPIKey, inApiCall, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        inAPIKeyLen = len(inAPIKey)
        while True:
            inApiCall = inApiCall + 1
            if inType == "jsonabnormal":
                time.sleep(1)
            inParam["serviceKey"] = inAPIKey[inApiCall%inAPIKeyLen]
            print("{} page scraping start apicall iter: {} / used {}".format(i,inApiCall,inParam["serviceKey"]))
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            
            response = requests.get(inUrl+queryParams)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                                   
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)
                
                jsondata = response.json()["response"]["body"]["detail2"]
                if( jsondata == []):
                    print("detail2 is empty")
                else:
                    jsondata["index"]=[0]
                    rowData = pd.DataFrame(jsondata)
                    emptyPd = pd.concat([emptyPd,rowData],axis=1)
#                     emptyPd = emptyPd.append(rowData)
                
                
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return [emptyPd,i,inApiCall]        
    except Exception as e:
            print(e)     

In [20]:

PAGEYN=0

In [ ]:

resultDfMerged = pd.DataFrame(columns=["ofcNm","rutNm","fctKdCd","fctKdNm","fctCfCd","fctCfNm","fcno","fctNm","lctNm","ccwDt","dgnBgDt","dgnEdDt","dgcoNm","cwkBgDt","cwkEdDt","cstCmpNm","cwkNm","totCtco","xtsLen","xtsHg","xtsSpanCo","xtsMaxSpanLen","ftptBdt","rdwyBdt","alBdt","uplnCrtrCo","godnCrtrCo","crtrStoCo","prmsPssnLwgt","upstSpanCmpsVl","bdsfAbscNm","railngAbscNm","brdgBrktAbscNm","xpsjAbscNm","curbAbscNm","dpltAbscVl","mngdAbscVl","dgldVlCn","upstMspFrmCdCn","upstSbSpanFrmAbrNmCn","upstBrdgBrktFrmCdCn","upstXpsjFrmCdCn","lwstPierFrmCdCn","lwstPierBsCdCn","lwstBrpsFrmCdCn","tnnlXtsLen","tnnlHg","tmnsSlpeVl","algnRdcvVl","dtsbXn","ivrtInstXn","rprsFormCdCn","rprsMnCnmtCdCn","rprsAssCnmtCdCn","rprsDrnFrmCdCn","rprsPtnnFrmNmCn"])
resultDf  = pd.DataFrame(columns=["ofcNm","rutNm","fctKdCd","fctKdNm","fctCfCd","fctCfNm","fcno","fctNm","lctNm","ccwDt","dgnBgDt","dgnEdDt","dgcoNm","cwkBgDt","cwkEdDt","cstCmpNm","cwkNm","totCtco","xtsLen","xtsHg","xtsSpanCo","xtsMaxSpanLen","ftptBdt","rdwyBdt","alBdt","uplnCrtrCo","godnCrtrCo","crtrStoCo","prmsPssnLwgt","upstSpanCmpsVl","bdsfAbscNm","railngAbscNm","brdgBrktAbscNm","xpsjAbscNm","curbAbscNm","dpltAbscVl","mngdAbscVl","dgldVlCn","upstMspFrmCdCn","upstSbSpanFrmAbrNmCn","upstBrdgBrktFrmCdCn","upstXpsjFrmCdCn","lwstPierFrmCdCn","lwstPierBsCdCn","lwstBrpsFrmCdCn","tnnlXtsLen","tnnlHg","tmnsSlpeVl","algnRdcvVl","dtsbXn","ivrtInstXn","rprsFormCdCn","rprsMnCnmtCdCn","rprsAssCnmtCdCn","rprsDrnFrmCdCn","rprsPtnnFrmNmCn"])

APICALL = 0 #추가
ptlcmnoListLen = len(ptlcmnoList)
pageList = [] #추가

for i in range(0, ptlcmnoListLen):
    apiNo = i%APIKEYLEN
#     fcnoValue = dfCombi.loc[i].fcno
    BASEPARAM={"serviceKey":APIKEY[0],"fcno": ptlcmnoList[i], "type":"json"}
    scrapyResult = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, APIKEY,APICALL,JSONKEY, DUMMY)
    resultDf = scrapyResult[0]
    pageList.append( scrapyResult[1] )
    APICALL = scrapyResult[2]
    if resultDf.empty: # 정상 데이터가 없는 경우
        print("No Data")
    else:
#         resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        print("i=",i," fcnoValue= ",ptlcmnoList[i])
        resultDf = resultDf.loc[:,~resultDf.columns.duplicated()] # 중복제거
        print(resultDf)
        resultDfMerged = resultDfMerged.append(resultDf) #.loc[0], ignore_index=True) 
#         print("----------------------------------------")
#         print(resultDf)
#resultDfMerged

1 page scraping start apicall iter: 1 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
i= 0  fcnoValue=  EFE3B42F11250191E0340003BA2D2647
       ofcnm     rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  광주국토관리사무소  일반국도29호선       1      교량       0      기타   

                               fcno fctnm             lctnm       ccwdt  \
0  EFE3B42F11250191E0340003BA2D2647   영벽교  전라남도 화순군 능주면 관영리  2002-06-01   

  dgnbgdt dgneddt    dgconm     cwkbgdt     cwkeddt      cstcmpnm cwknm  \
0    None    None  (주)우대기술단  1998-07-01  2002-06-01  임광토건(주)외 3개사  None   

   totctco  index  xtslen  xtshg  xtsspanco  xtsmaxspanlen  ftptbdt  rdwybdt  \
0  639,200      0      13    9.6          1             13      0.8     18.5   

   albdt  uplncrtrco  godncrtrco  crtrstoco  prmspssnlwgt upstspancmpsvl  \
0   19.4           2           2          4            43        1@13=13   

  bdsfabscnm railngabscnm brdgbrktabscnm xpsjabscnm curbabscnm dpltabscvl  \
0       아스팔트           강재           None     

1 page scraping start apicall iter: 8 / used 0974320D-C689-4543-8464-10DE1E5505A9
0 no pageNo
i= 7  fcnoValue=  01773768DCD3A3239547956E81959B9E
       ofcnm     rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  대구국토관리사무소  일반국도28호선       1      교량       0      기타   

                               fcno fctnm                          lctnm  \
0  01773768DCD3A3239547956E81959B9E   일신교  경상북도 의성군 다인면 덕미리 ~ 단북면 연제리 일원   

        ccwdt     dgnbgdt     dgneddt                 dgconm     cwkbgdt  \
0  2018-09-04  2009-12-08  2012-12-22  (주)도우엔지니어즈,(주)선진엔지니어링  2014-02-01   

      cwkeddt         cstcmpnm          cwknm  totctco  index  xtslen xtshg  \
0  2014-11-30  남 진 건 설 주 식 회 사  다인-비안1 국도건설공사  223,173      0      13  None   

   xtsspanco  xtsmaxspanlen  ftptbdt  rdwybdt  albdt  uplncrtrco  godncrtrco  \
0          1             13        0       35     35           2           2   

   crtrstoco  prmspssnlwgt upstspancmpsvl bdsfabscnm railngabscnm  \
0          4            58        1@13=13  

0             목조식             기타             역T형  
1 page scraping start apicall iter: 15 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
i= 14  fcnoValue=  364D9EE56B7F3609E054B099289CEB28
        ofcnm    rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  의정부국토관리사무소  일반국도3호선       1      교량       1      1종   

                               fcno    fctnm        lctnm       ccwdt  \
0  364D9EE56B7F3609E054B099289CEB28  회암천교(상)  경기도 양주시 회암동  2015-12-31   

      dgnbgdt     dgneddt    dgconm     cwkbgdt     cwkeddt cstcmpnm  \
0  1997-12-15  1998-12-31  ㈜만영엔지니어링  1999-05-26  2015-12-31    한신공영㈜   

                          cwknm    totctco  index  xtslen  xtshg  xtsspanco  \
0  의정부시관내 국도대체 우회도로(자금-회천) 건설공사  4,589,000      0     145     15          3   

   xtsmaxspanlen  ftptbdt  rdwybdt  albdt  uplncrtrco  godncrtrco  crtrstoco  \
0             55        0       15     15           3           0          3   

   prmspssnlwgt upstspancmpsvl bdsfabscnm railngabscnm brdgbrktabscnm  \


1 page scraping start apicall iter: 21 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
i= 20  fcnoValue=  8EF3E29E0D152B45E054B099289CEB28
       ofcnm     rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  수원국토관리사무소  일반국도42호선       1      교량       2      2종   

                               fcno fctnm            lctnm       ccwdt  \
0  8EF3E29E0D152B45E054B099289CEB28  상하2교  경기도 용인시 기흥구 상하동  2018-12-31   

  dgnbgdt dgneddt dgconm cwkbgdt cwkeddt cstcmpnm cwknm totctco  index  \
0    None    None   None    None    None     None  None       0      0   

   xtslen  xtshg  xtsspanco  xtsmaxspanlen ftptbdt rdwybdt  albdt  uplncrtrco  \
0     265      2          7             45    None    None     26          22   

   godncrtrco  crtrstoco prmspssnlwgt upstspancmpsvl bdsfabscnm railngabscnm  \
0           2          4         None           None       None         None   

  brdgbrktabscnm xpsjabscnm curbabscnm dpltabscvl mngdabscvl dgldvlcn  \
0           None       None       None   

1 page scraping start apicall iter: 27 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
i= 26  fcnoValue=  7E22297EDB0B0387E0440003BA816705
       ofcnm     rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  예산국토관리사무소  일반국도21호선       1      교량       1      1종   

                               fcno fctnm             lctnm       ccwdt  \
0  7E22297EDB0B0387E0440003BA816705   창동교  충청남도 보령시 남포면 창동리  2009-12-31   

  dgnbgdt dgneddt dgconm     cwkbgdt     cwkeddt  cstcmpnm  \
0    None    None  (주)삼안  2002-02-08  2009-12-31  코오롱건설(주)   

                              cwknm    totctco  index  xtslen  xtshg  \
0  국도21호선 보령시 국도대체 우회도로(화산-옥동) 건설공사  6,857,395      0     370      5   

   xtsspanco  xtsmaxspanlen  ftptbdt  rdwybdt  albdt  uplncrtrco  godncrtrco  \
0          8             55      0.9       20   20.9           2           2   

   crtrstoco  prmspssnlwgt upstspancmpsvl bdsfabscnm railngabscnm  \
0          4            43      1@45+7@55       아스팔트         콘크리트   

  brdgbrktabscn

...

1 page scraping start apicall iter: 91 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
i= 90  fcnoValue=  01701E3E1D70A597CD1A42BC68C2A732
       ofcnm    rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  순천국토관리사무소  일반국도2호선       3    지하차도       2      2종   

                               fcno   fctnm          lctnm       ccwdt  \
0  01701E3E1D70A597CD1A42BC68C2A732  정산지하차도  전라남도 광양시 성황동   2019-12-24   

      dgnbgdt     dgneddt    dgconm     cwkbgdt     cwkeddt cstcmpnm  \
0  2009-03-13  2010-12-13  한국해외기술공사  2011-12-28  2019-12-24  GS건설(주)   

                      cwknm    totctco  index  xtslen  tnnlhg  ftptbdt  \
0  광양시 국도대체우회도로(중군-진정) 건설공사  2,356,000      0     110     7.3      4.6   

   rdwybdt  albdt  uplncrtrco  godncrtrco  totcrtrco  tmnsslpevl  algnrdcvvl  \
0       18   22.6           2           2          4         1.8         850   

  rprsformcdcn rprsmncnmtcdcn rprsasscnmtcdcn drnfrmcdcn  
0         None           None            None        배수식  
1 page scraping 

1 page scraping start apicall iter: 98 / used 0974320D-C689-4543-8464-10DE1E5505A9
0 no pageNo
i= 97  fcnoValue=  EFE3B42F11950191E0340003BA2D2647
       ofcnm     rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  전주국토관리사무소  일반국도21호선       1      교량       2      2종   

                               fcno   fctnm             lctnm       ccwdt  \
0  EFE3B42F11950191E0340003BA2D2647  탑천교(하)  전라북도 군산시 대야면 광교리  2002-06-01   

  dgnbgdt dgneddt       dgconm     cwkbgdt     cwkeddt cstcmpnm         cwknm  \
0    None    None  (주)도화종합기술공사  1999-12-01  2002-06-01  (주)두산건설  대야-공덕 도로건설공사   

     totctco  index  xtslen  xtshg  xtsspanco  xtsmaxspanlen  ftptbdt  \
0  3,993,000      0   120.3      9          4             30      0.7   

   rdwybdt  albdt  uplncrtrco  godncrtrco  crtrstoco  prmspssnlwgt  \
0     11.4   12.1           0           2          2            40   

   upstspancmpsvl bdsfabscnm railngabscnm brdgbrktabscnm xpsjabscnm  \
0  4 @ 30 = 120.3       아스팔트         콘크리트            고무재   

0            None           None             역T형  
1 page scraping start apicall iter: 105 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
i= 104  fcnoValue=  E673B64271321C52E034080020A9316A
        ofcnm     rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  의정부국토관리사무소  일반국도47호선       1      교량       3      3종   

                               fcno fctnm             lctnm       ccwdt  \
0  E673B64271321C52E034080020A9316A  부평2교  경기도 남양주시 진접읍 팔야리  1997-12-30   

  dgnbgdt dgneddt    dgconm     cwkbgdt     cwkeddt  cstcmpnm           cwknm  \
0    None    None  고려건설주식회사  1993-09-01  1997-12-30  강산건설주식회사  진접~신팔 확-포장2차공사   

   totctco  index  xtslen  xtshg  xtsspanco  xtsmaxspanlen  ftptbdt  rdwybdt  \
0  152,382      0      40      5          3             15        0     18.5   

   albdt  uplncrtrco  godncrtrco  crtrstoco  prmspssnlwgt upstspancmpsvl  \
0   19.5           2           2          4            40   2@12.5+15=40   

  bdsfabscnm railngabscnm brdgbrktabscnm xpsjabscnm c

1 page scraping start apicall iter: 111 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
i= 110  fcnoValue=  08F82CB466A54820E0440003BA8152A1
       ofcnm     rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  예산국토관리사무소  일반국도32호선       1      교량       3      3종   

                               fcno   fctnm             lctnm       ccwdt  \
0  08F82CB466A54820E0440003BA8152A1  거산I.C교  충청남도 당진시 신평면 거산리  2005-11-22   

      dgnbgdt     dgneddt   dgconm     cwkbgdt     cwkeddt cstcmpnm  \
0  1997-06-25  1998-09-16  보람엔지니어링  1999-04-30  2005-11-22  경남기업(주)   

                cwknm  totctco  index  xtslen  xtshg  xtsspanco  \
0  신평~합덕간 도로확장 및 포장공사  801,000      0    76.2    8.2          2   

   xtsmaxspanlen  ftptbdt  rdwybdt  albdt  uplncrtrco  godncrtrco  crtrstoco  \
0           38.1        0        7    7.4           0           1          1   

   prmspssnlwgt upstspancmpsvl bdsfabscnm railngabscnm brdgbrktabscnm  \
0            43    2@38.1=76.2       아스팔트         콘크리트             

1 page scraping start apicall iter: 118 / used 0974320D-C689-4543-8464-10DE1E5505A9
0 no pageNo
i= 117  fcnoValue=  AC795B75D1CB0FD7E0440003BA816705
       ofcnm     rutnm fctkdcd fctkdnm fctcfcd fctcfnm  \
0  남원국토관리사무소  일반국도17호선       1      교량       3      3종   

                               fcno fctnm         lctnm       ccwdt  \
0  AC795B75D1CB0FD7E0440003BA816705   신정교  전라북도 남원시 신정동  2007-12-30   

      dgnbgdt     dgneddt    dgconm     cwkbgdt     cwkeddt cstcmpnm  \
0  1996-12-01  1997-12-01  새길엔지니어링㈜  1999-05-12  2007-12-30  중흥건설(주)   

                           cwknm    totctco  index  xtslen  xtshg  xtsspanco  \
0  남원시 관내 국도대체 우회도로 (주생~광치) 건설공사  1,291,000      0      40    4.6          1   

   xtsmaxspanlen  ftptbdt  rdwybdt  albdt  uplncrtrco  godncrtrco  crtrstoco  \
0             40        1       20     21           2           2          4   

   prmspssnlwgt upstspancmpsvl bdsfabscnm railngabscnm brdgbrktabscnm  \
0            40           None       None         N

In [28]:
resultDfMerged

,ofcNm,rutNm,fctKdCd,fctKdNm,fctCfCd,fctCfNm,fcno,fctNm,lctNm,ccwDt,dgnBgDt,dgnEdDt,dgcoNm,cwkBgDt,cwkEdDt,cstCmpNm,cwkNm,totCtco,xtsLen,xtsHg,xtsSpanCo,xtsMaxSpanLen,ftptBdt,rdwyBdt,alBdt,uplnCrtrCo,godnCrtrCo,crtrStoCo,prmsPssnLwgt,upstSpanCmpsVl,bdsfAbscNm,railngAbscNm,brdgBrktAbscNm,xpsjAbscNm,curbAbscNm,dpltAbscVl,mngdAbscVl,dgldVlCn,upstMspFrmCdCn,upstSbSpanFrmAbrNmCn,upstBrdgBrktFrmCdCn,upstXpsjFrmCdCn,lwstPierFrmCdCn,lwstPierBsCdCn,lwstBrpsFrmCdCn,tnnlXtsLen,tnnlHg,tmnsSlpeVl,algnRdcvVl,dtsbXn,ivrtInstXn,rprsFormCdCn,rprsMnCnmtCdCn,rprsAssCnmtCdCn,rprsDrnFrmCdCn,rprsPtnnFrmNmCn,ofcnm,rutnm,fctkdcd,fctkdnm,fctcfcd,fctcfnm,fctnm,lctnm,ccwdt,dgnbgdt,dgneddt,dgconm,cwkbgdt,cwkeddt,cstcmpnm,cwknm,totctco,index,xtslen,xtshg,xtsspanco,xtsmaxspanlen,ftptbdt,rdwybdt,albdt,uplncrtrco,godncrtrco,crtrstoco,prmspssnlwgt,upstspancmpsvl,bdsfabscnm,railngabscnm,brdgbrktabscnm,xpsjabscnm,curbabscnm,dpltabscvl,mngdabscvl,dgldvlcn,upstmspfrmcdcn,upstsbspanfrmabrnmcn,upstbrdgbrktfrmcdcn,upstxpsjfrmcdcn,lwstpierfrmcdcn,lwstpierbscdcn,lwstbrpsfrmcdcn
0,NaN,NaN,NaN,NaN,NaN,NaN,EFE3B42F11250191E0340003BA2D2647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,광주국토관리사무소,일반국도29호선,1,교량,0,기타,영벽교,전라남도 화순군 능주면 관영리,2002-06-01,None,None,(주)우대기술단,1998-07-01,2002-06-01,임광토건(주)외 3개사,None,"639,200",0.0,13.0,9.6,1.0,13.0,0.8,18.5,19.4,2.0,2,4.0,43.0,1@13=13,아스팔트,강재,None,None,콘크리트,None,None,DB-24,라멘교,None,None,None,None,None,라멘식
0,NaN,NaN,NaN,NaN,NaN,NaN,F2DF88947888084CE0340003BA816705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,순천국토관리사무소,일반국도2호선,1,교량,3,3종,상리교,전라남도 장흥군 장흥읍 해당리,2003-12-30,1995-05-17,1996-06-30,동부엔지니어링(주),1996-12-02,2003-12-30,경남기업(주),강진-장흥간 도로확장 및 포장공사,"1,130,000",0.0,30.0,5.0,1.0,30.0,0.8,19.4,20.3,2.0,2,4.0,43.0,1@30=30,콘크리트,콘크리트,강재,강재,콘크리트,RC,PC,DB-24,PSCI형교,PSCI형교,포트받침,TransFlex조인트,기타,미상,역T형
0,NaN,NaN,NaN,NaN,NaN,NaN,7D6B67A667A46EDCE0440003BA816705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,영주국토관리사무소,일반국도34호선,1,교량,0,기타,신석육교,경상북도 안동시 남선면 이천리,2009-12-24,1999-04-06,2000-09-16,(주)동아기술공사,2001-12-29,2009-12-23,(주)한진중공업,안동시관내국도대체우회도로(신석-용상)건설공사,"553,269",0.0,15.0,6.5,1.0,15.0,0.0,24.0,24.9,2.0,2,4.0,43.0,1@15=15M,아스팔트,콘크리트,None,None,None,RC,None,DB-24,라멘교,None,없음,신축이음 없음,라멘식,강재말뚝기초,라멘식
0,NaN,NaN,NaN,NaN,NaN,NaN,4F845995A51F2E66E0440003BA816705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,광주국토관리사무소,일반국도18호선,1,교량,0,기타,남리3교,전라남도 해남군 황산면 호동리,2007-12-26,None,None,도화종합기술공사,None,2007-12-26,송촌건설,문내-황산간 도로확장 및 포장공사,"355,590",0.0,16.8,7.8,1.0,16.8,0.0,6.5,7.5,1.0,None,4.0,40.0,1@16.8,아스팔트,콘크리트,None,None,None,RC,Steel,DB-24,라멘교,None,None,None,None,None,None


In [30]:
resultDfMerged = resultDfMerged.dropna(how='all', axis=1)
resultDfMerged

,fcno,ofcnm,rutnm,fctkdcd,fctkdnm,fctcfcd,fctcfnm,fctnm,lctnm,ccwdt,dgnbgdt,dgneddt,dgconm,cwkbgdt,cwkeddt,cstcmpnm,cwknm,totctco,index,xtslen,xtshg,xtsspanco,xtsmaxspanlen,ftptbdt,rdwybdt,albdt,uplncrtrco,godncrtrco,crtrstoco,prmspssnlwgt,upstspancmpsvl,bdsfabscnm,railngabscnm,brdgbrktabscnm,xpsjabscnm,curbabscnm,dpltabscvl,mngdabscvl,dgldvlcn,upstmspfrmcdcn,upstsbspanfrmabrnmcn,upstbrdgbrktfrmcdcn,upstxpsjfrmcdcn,lwstpierfrmcdcn,lwstpierbscdcn,lwstbrpsfrmcdcn
0,EFE3B42F11250191E0340003BA2D2647,광주국토관리사무소,일반국도29호선,1,교량,0,기타,영벽교,전라남도 화순군 능주면 관영리,2002-06-01,None,None,(주)우대기술단,1998-07-01,2002-06-01,임광토건(주)외 3개사,None,"639,200",0.0,13.0,9.6,1.0,13.0,0.8,18.5,19.4,2.0,2,4.0,43.0,1@13=13,아스팔트,강재,None,None,콘크리트,None,None,DB-24,라멘교,None,None,None,None,None,라멘식
0,F2DF88947888084CE0340003BA816705,순천국토관리사무소,일반국도2호선,1,교량,3,3종,상리교,전라남도 장흥군 장흥읍 해당리,2003-12-30,1995-05-17,1996-06-30,동부엔지니어링(주),1996-12-02,2003-12-30,경남기업(주),강진-장흥간 도로확장 및 포장공사,"1,130,000",0.0,30.0,5.0,1.0,30.0,0.8,19.4,20.3,2.0,2,4.0,43.0,1@30=30,콘크리트,콘크리트,강재,강재,콘크리트,RC,PC,DB-24,PSCI형교,PSCI형교,포트받침,TransFlex조인트,기타,미상,역T형
0,7D6B67A667A46EDCE0440003BA816705,영주국토관리사무소,일반국도34호선,1,교량,0,기타,신석육교,경상북도 안동시 남선면 이천리,2009-12-24,1999-04-06,2000-09-16,(주)동아기술공사,2001-12-29,2009-12-23,(주)한진중공업,안동시관내국도대체우회도로(신석-용상)건설공사,"553,269",0.0,15.0,6.5,1.0,15.0,0.0,24.0,24.9,2.0,2,4.0,43.0,1@15=15M,아스팔트,콘크리트,None,None,None,RC,None,DB-24,라멘교,None,없음,신축이음 없음,라멘식,강재말뚝기초,라멘식
0,4F845995A51F2E66E0440003BA816705,광주국토관리사무소,일반국도18호선,1,교량,0,기타,남리3교,전라남도 해남군 황산면 호동리,2007-12-26,None,None,도화종합기술공사,None,2007-12-26,송촌건설,문내-황산간 도로확장 및 포장공사,"355,590",0.0,16.8,7.8,1.0,16.8,0.0,6.5,7.5,1.0,None,4.0,40.0,1@16.8,아스팔트,콘크리트,None,None,None,RC,Steel,DB-24,라멘교,None,None,None,None,None,None


In [ ]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

In [ ]:
resultDfMerged.shape

In [ ]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
ptlcmnoList = orgParam + ptlcmnoList

paramList = [ptlcmnoList]
# 최종 파라미터 저장
cf.saveparam(paramList, SITENAME,DATANAME,SERVICENAME)

## 응답 칼럼 정리하여 DataFrame의 칼럼으로 선언하기 위한 사전작업

In [ ]:
strTemp = "ofcNm, rutNm, fctKdCd, fctKdNm, fctCfCd, fctCfNm, fcno, fctNm, lctNm, ccwDt, dgnBgDt, dgnEdDt, dgcoNm, cwkBgDt, cwkEdDt, cstCmpNm, cwkNm, totCtco ,xtsLen, xtsHg, xtsSpanCo, xtsMaxSpanLen, ftptBdt, rdwyBdt, alBdt, uplnCrtrCo, godnCrtrCo, crtrStoCo, prmsPssnLwgt, upstSpanCmpsVl, bdsfAbscNm, railngAbscNm, brdgBrktAbscNm, xpsjAbscNm, curbAbscNm, dpltAbscVl, mngdAbscVl, dgldVlCn, upstMspFrmCdCn, upstSbSpanFrmAbrNmCn, upstBrdgBrktFrmCdCn, upstXpsjFrmCdCn, lwstPierFrmCdCn, lwstPierBsCdCn, lwstBrpsFrmCdCn ,tnnlXtsLen, tnnlHg, ftptBdt, rdwyBdt, alBdt, uplnCrtrCo, godnCrtrCo, crtrStoCo, tmnsSlpeVl, algnRdcvVl, dtsbXn, ivrtInstXn, rprsFormCdCn, rprsMnCnmtCdCn, rprsAssCnmtCdCn, rprsDrnFrmCdCn, rprsPtnnFrmNmCn,tnnlXtsLen, tnnlHg, ftptBdt, rdwyBdt, alBdt, uplnCrtrCo, godnCrtrCo, crtrStoCo, tmnsSlpeVl, algnRdcvVl, dtsbXn, ivrtInstXn, rprsFormCdCn, rprsMnCnmtCdCn, rprsAssCnmtCdCn, rprsDrnFrmCdCn, rprsPtnnFrmNmCn"
strSum = ""
strTemp = strTemp.split(",")
stripped = [s.strip() for s in strTemp]

In [ ]:
strSum=""
resultStripped = []
for value in stripped: 
    if value not in resultStripped: 
        resultStripped.append(value)

for i in range(0,len(resultStripped)):
    strme = '\"'+resultStripped[i]+'\"'+','
    strSum += strme
print(strSum)

In [ ]:
df = pd.DataFrame(columns=["ofcNm","rutNm","fctKdCd","fctKdNm","fctCfCd","fctCfNm","fcno","fctNm","lctNm","ccwDt","dgnBgDt","dgnEdDt","dgcoNm","cwkBgDt","cwkEdDt","cstCmpNm","cwkNm","totCtco","xtsLen","xtsHg","xtsSpanCo","xtsMaxSpanLen","ftptBdt","rdwyBdt","alBdt","uplnCrtrCo","godnCrtrCo","crtrStoCo","prmsPssnLwgt","upstSpanCmpsVl","bdsfAbscNm","railngAbscNm","brdgBrktAbscNm","xpsjAbscNm","curbAbscNm","dpltAbscVl","mngdAbscVl","dgldVlCn","upstMspFrmCdCn","upstSbSpanFrmAbrNmCn","upstBrdgBrktFrmCdCn","upstXpsjFrmCdCn","lwstPierFrmCdCn","lwstPierBsCdCn","lwstBrpsFrmCdCn","tnnlXtsLen","tnnlHg","tmnsSlpeVl","algnRdcvVl","dtsbXn","ivrtInstXn","rprsFormCdCn","rprsMnCnmtCdCn","rprsAssCnmtCdCn","rprsDrnFrmCdCn","rprsPtnnFrmNmCn"])
df